<h1><span style="color:red">Please read this very carefully! </span></h1>

In order to setup your own experiments, you need to download remote files to your linux disk image in the collaboratory environment. As data for your user account is NOT reset when you close or reload the HBP, you have to be very careful how you organize & structure your data. In order to help you with that we create a unique working directory for each molecular use case you run.

Please be also aware that we switch current working directories in this use case. That means that you have to restart and clear all output in order to go back to your starting directory. 

# Analyse the results of a Brownian dynamics simulation for calculating protein-protein association rate constants

**Aim:** This use case shows how to compute a bimolecular association rate constant from Brownian dynamics simulations of the diffusional association of two proteins.

**Version:** 1.0 (March 2019)

**Contributors:**  Neil Bruce, Lukas Adam, Stefan Richter, Rebecca Wade (HITS, Heidelberg, Germany)

**Contact:** [mcmsoft@h-its.org](mailto:mcmsoft@h-its.org)


## Setting up your environment

### Check that all required python packages are installed and working

In [ ]:
# Check that required packages are installed
! pip install --upgrade "hbp-service-client" 
! pip install wget python-magic

In [ ]:
# Import python packages used in this notebook
try: 
    import os, wget, warnings, subprocess, datetime, magic
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from pylab import rcParams
    from hbp_service_client.storage_service.client import Client
except: 
    print("Importing python modules failed.\n\tThere is a problem with the python environment!")
else:
    print("Python environment set correctly")

### Check that SDA is available

In [ ]:
# Check that SDA is environment variable is set
try:
    sdaDir = os.environ['SDAHOME']
except:
    print("Unable to find SDA. There is a problem")
else:
    print("SDA is installed in %s" % sdaDir)
    
# Check that bootstrap script used in this use case is available
bootScr = os.path.join(sdaDir, 'auxi', 'Bootstrap_multiCPU.py')
if os.path.isfile(bootScr):
    print ("Bootstrap file found: %s" % bootScr)
else:
    print("Bootstrap file is not found. There is an error in the SDA installation.")

### Set up local directory and download required files

In [ ]:
# Create a local directories
try:
    homeDir = os.environ['HOME']
except:
    print("Error in environment")

else:
    workDir = os.path.join(homeDir, 'work')
    if not os.path.isdir(workDir):
        try:
            os.mkdir(workDir)
        except:
            print("unable to make working directory")
    
    # Make a new directory to run the use case in. 
    # If directory already exists, add a number to make a unique name
    baseDir = 'sdaAnalysis'
    dirIter = 0
    useCaseDir = os.path.join(workDir, baseDir)
    print(useCaseDir)
    
    if os.path.exists(useCaseDir):
        while os.path.exists(useCaseDir):
            dirIter += 1
            useCaseDir = os.path.join(workDir, baseDir + '.' + str(dirIter))            
    
    try:
        os.mkdir(useCaseDir)
    except:
        print("Failed to make use case working directory")
    else:
        print("Working directory for current use case: %s" % useCaseDir)


In [ ]:
# Download SDA bootstrap output file from CSCS storage for analysis
try:
    print("Downloading SDA bootstrap output file from CSCS storage area")
    fileUrl= 'https://object.cscs.ch/v1/AUTH_c0a333ecf7c045809321ce9d9ecdfdea/SGA2_molecular_signalling_cascades/data/Predicted_association_rate_constants_of_G_proteins_to_AC5_complexes/bootstrap_logfiles/holo_kf3_AC5-Golf_Gi/snap_0/rep_1/assoc_bootstrap.log'
    wget.download(fileUrl, useCaseDir)
except:
    print("Error downloading bootstrap output file from CSCS storage")
else:
    print("Sucessfully downloaded the bootstrap output file from CSCS storage")


### Set up collab storage for saving data at end of calculation

In [ ]:
#Find your own collab storage path
collab_path = get_collab_storage_path()
print(collab_path)
storage_client = Client.new(oauth.get_token())

**If all of the above cells have completed without error, your python environment and local directories are set up correctly, and you have all the files required for this use case.**

# Simulation of Diffusional Association - Analysis # 

In this use case, we analyse the results of an [SDA](https://collab.humanbrainproject.eu/#/collab/19/nav/2108?state=software,SDA) simulation of the association of the G<sub>olf</sub> α-subunit to the enzyme adenylyl cyclase 5 (AC5) [*An upcoming BSP use case will explain how to set up and run these simulations*]. This use case will explain:
* The functionality provided by SDA to calculate the biomolecular rate of Gα<sub>olf</sub> associating to AC5 by analysing the contact formation between the two protein species during a set of 50&nbsp;000 completed Brownian dynamics simulation trajectories.
* The use of a bootstrapping procedure to estimate the error in the predicted rates.
* The use of matplotlib to plot the dependence of association rates on how we define the end point of association based on contact formation.

** Recap: How is protein diffusion modeled in Brownian dynamics simulations? **

As the interaction of large biomolecules in solution is a highly complex problem with many degrees of freedom, the diffusional motion of proteins in Brownian dynamics simulations is often modelled by assuming the proteins to be rigid bodies. Furthermore, their interactions with surrounding water molecules are modeled implicitly (using random forces that mimic the collisions between solutes and water molecules). These assumptions not only speed up the calculation of the systematic forces by allowing the use of precomputed interaction grids, but also allow for larger simulation time steps, as faster vibrational motions are removed. At each simulation step, the position $r$ of solute $i$ within a system of $N$ solutes is propagated using the Ermak-McCammon equation:

$$ \Delta r_{i} = \Delta t\sum_{j=1}^{N}\left ( \frac{\partial \widehat{D}_{ij}}{\partial r_{j}} + \frac{\widehat{D}_{ij}}{k_{B}T} \cdot F_{i} \right ) + R_{i} $$

where $k_{B}$ and $T$ are the Boltzmann constant and simulation temperature, respectively, and $F_{i}$ is the force acting on solute $i$ due to its interactions with all other solutes. $\mathbf{\widehat{D}}$ is the diffusion tensor for the current configuration of the system, and $R_{i}$ is a random displacement vector obtained from the factorisation of $\mathbf{\widehat{D}}$. The dependence of the diffusion of one solute on the diffusion of all other solutes is due to hydrodynamic interactions, where solutes are able to feel the flow fields in solution created by the other solutes. 

Modelling hydrodynamic interactions is extremely computationally expensive, due to the need to factorise $\mathbf{\widehat{D}}$ at every time step. In dilute conditions, as we use rigid solutes, we can assume these interactions are negligible, simplifying the equation and removing the need for matrix factorisation:

$$ \Delta r_{i} = \frac{\Delta t}{k_{B}T} D_{i}^{T} \cdot F_{i} + R_{i} $$

where $ D_{i}^{T}$ is the infinite dilution translational diffusion coefficient of solute $i$, and $R_{i}$ is a random vector sampled from a Gaussian distribution with mean zero and a variance of $\left \langle R_{i}^{2} \right \rangle = 6D_{i}^{T} \Delta t$. A similar equation is used to propagate the rotational motion of the solutes based on the torques acting on them.

For information on the calculation of the contributions to the force $F_{i}$, see [Martinez et al (2015), Journal of Computational Chemistry, 36, 1631–1645](https://mcm.h-its.org/sda7/doc/doc_sda7/jcc23971.pdf).


** Prediction of bimolecular association rate constant **

SDA calculates bimolecular association rate constants using the [Northrup, Allison and McCammon](https://doi.org/10.1063/1.446900) method. Here we run many thousands of Brownian dynamics trajectories, each beginning with the two solutes (molecules, in this case proteins) at a separation $b$ (shown by the b-surface in the figure below). In each trajectory, we then simulate the diffusion of one solute relative to another, until the solutes are separated by a larger distance $c$ (shown by the c-surface in the figure below).

![bsurface_csurface](https://projects.h-its.org/mcmsoft/hbpdata/sda_bsurface_csurface.gif)

During each trajectory, we monitor the formation of polar contacts present in the native state of the complex formed by the two solutes. If we define a set of reaction criteria that says a reactive encounter occurs when $N$ native contacts are within a certain distance, we can calculate the fraction $\beta$ of our trajectories in which these criteria are met, and estimate the bimolecular association rate constant $k_{on}$ from

$$k_{on} = \frac{k_b\beta}{1-(1-\beta)\frac{k_b}{k_c}}$$

where $k_b$ and $k_c$ are the rates at which the solutes diffuse to separations $b$ and $c$, respectively. If we assume the interactions between the solutes do not depend on their relative orientations at these large distances, these rates can be calculated analytically.


** The assoc_bootstrap.log file downloaded from CSCS storage **

In the set up cells run at the top of this notebook, we downloaded a file called *assoc_bootstrap.log*. This file gives a concise representation of the results of 50&nbsp;000 Brownian dynamics simulation trajectories. The header to the file gives some information about the simulations that have been run.


nb_contact | nb_wind | first_win | width_win | b_rate       | c_rate       | nrun  |
------------|----------|-----------|-----------|--------------|--------------|-------|
 4          | 35       | 3.00      | 0.50      | 0.231552E+11 | 0.694655E+11 | 50000 |

In each trajectory, we monitor the formation of up to *nb_contact* native contacts over *nb_wind* distance windows of *width_win* &#8491;, starting from a distance of *first_win* &#8491;. The *b_rate* and *c_rate*, correspond to $k_b$ and $k_c$ in the equation above. In total, *nrun* trajectories were run. The rest of the file is separated into sections corresponding to each number of contacts that we monitor. Each line represents a single trajectory, and contains a number showing the closest window at which that number of contacts was obtained during the trajectory.


** What is bootstrapping? **

Bootstrapping is defined as any test or metric that relies on random resampling with replacement. It allows us to assign measures of accuracy (e.g. confidence intervals, standard deviations or standard errors) to sample estimates. By randomly resampling the data, one can approximate the underlying statistical distribution. 

The SDA script *Bootstrap_multiCPU.py* analyses the *assoc_bootstrap.log* file to predict rate constants for various encounter definitions. It then performs bootstrapping to estimate the standard deviation in the prediction across a set of resampled trajectories.

** Outline of the analysis below**

During the 50&nbsp;000 trajectories described in the log file, we monitor the formation of up to 4 native contacts. In the cells below, we perform bootstrapping by randomly resampling 200 sets of 50&nbsp;000 trajectories. From there, we will extract the values and convert them to a pandas dataframe, which will then be plotted with matplotlib, and we save summary data to a set of output files. 

### Run bootstrapping analysis on SDA bootstrap output file

In [ ]:
warnings.filterwarnings('ignore')
%matplotlib inline

# Move to the working directory for this use case
os.chdir(useCaseDir)
 
assoc_bootstrap_log = os.path.join(useCaseDir, 'assoc_bootstrap.log')
print(assoc_bootstrap_log, bootScr)

In [ ]:
# Run the bootstrap analysis script on the SDA output file

# Output from the scripts is saved into a list of pandas data frames for 
# different number of contacts

# Create a dataframe for current contact number
def kin2pandas(contact):
    return pd.DataFrame([list(filter(None, i.split(' '))) for i in list(filter(None, contact))])

# Create list of contacts
def get_contacts(kin):
    indices = [0] + [i for i, x in enumerate(kin) if x == ""] + [len(kin)]
    contacts = []
    for i in range(len(indices)-1):
        contacts.append(kin[indices[i]:indices[i+1]])
    contacts = [kin2pandas(contact) for contact in contacts]
    return contacts

# Run Bootstrap analysis script on SDA output file
def extract_kinetics(sda_bootstrap, assoc_bootstrap_log):
    cmd = ['python', sda_bootstrap, '-i', assoc_bootstrap_log]
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    p = list(p.communicate())[0].decode("utf-8").split('\n')
    p = [i.strip() for i in p]
    p = p[p.index('# distance  average  std.deviation')+1:]
    kin = get_contacts(p)
    return kin
       
kin = extract_kinetics(bootScr, assoc_bootstrap_log)

In [ ]:
# Loading dataframe for one contact to check rates have been calculated
kin[0].head()

In [ ]:
def write_summary(kin):
    contactsFiles = []
    for i in range(len(kin) - 1):
        nContacts = i+1
        file = 'contacts_' + str(nContacts) + '.dat'
        fr = kin[i]
        fr.columns = ['#Dist', 'Rate M-1s-1', 'std dev']
        fr.to_csv(file, index=False, sep='\t')
        contactsFiles.append(file)
    return contactsFiles

# Write data for all numbers of contacts to files
contactsFiles = write_summary(kin)

In [ ]:
#from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

def plot_errorbar_kinetics(kinetics, path_to_save_file, filename):
    lines = []
    labels = []
    for index in range(len(kinetics)-1):
        ax = plt.axes()
        kinetics[index].columns = ['distance', 'average', 'std.deviation']
        kinet = kinetics[index].astype('float32')
        x = kinet.loc[:, 'distance'].tolist()
        y = kinet.loc[:,'average']
        yerr = kinet.loc[:,'std.deviation']
        line = ax.errorbar(x, y, yerr=yerr, fmt='o')
        ax.set_yscale("log", nonposy='clip')
        labels.append('Contacts: ' + str(index+1))
        lines.append(line)
    plt.legend(lines, labels)
    plt.title('SDA Bootstrapping: Contact distance vs. average association rate')
    plt.xlabel('distance (A)')
    plt.ylabel('average association rate (M-1 s-1)')
    plt.savefig(os.path.join(path_to_save_file, filename))
    plt.show()

In [ ]:
plot_errorbar_kinetics(kin, useCaseDir, 'SDA_bootstrap_ratecurve.png')

In the figure above, the average association rate constant is plotted on a logarithmic scale against the contact distance in &#8491; when 1 to 4  contacts between the two solutes, as defined by the reaction criteria, are shorter than the contact distance. One can see that the association rate constant decreases the closer the solutes come to each other.  It also decreases as the number of contacts that have to be satisfied increases. These trends can be explained by the decreasing number of ways the two solutes can arrange with respect to each other as the reaction criteria become more stringent.  From this plot, the association rate constant for a given reaction criterion can be extracted and used for further modeling. 

In this example, we estimate the the rate constant of Gα<sub>olf</sub> associating to AC5 using a reaction criterion of two contacts at a distance of 6 &#8491;.




In [ ]:
# Read association rate from data frame
row = kin[1].loc[kin[1]['distance'] == '6.000']
rate =  row['average'].values[0]
stddev = row['std.deviation'].values[0]

print('Association rate constant = %9.2e +/-%9.2e M-1s-1' % (float(rate), float(stddev)))

### Saving your data to the collab storage area 

In the following cells, your data will be moved to the storage area for your collab, and the local working directory will be cleaned.

In [ ]:
# Set up a timestamped directory name for saving results to the storage area
baseStorageDir = 'sdaAnalysisOutput_'
timestamp = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
storageDir = os.path.join(collab_path, baseStorageDir + timestamp)
try:
    print('Creating storage directory: %s' % storageDir)
    storage_client.mkdir(storageDir)
except:
    print('There was an error creating the storage directory')
else:
    # Copy files to the storage area and remove the local file
    cleanDir = True
    for fName in os.listdir(useCaseDir):
        localFile = os.path.join(useCaseDir, fName)
        storageFile = os.path.join(storageDir, fName)
        fType =  magic.Magic(mime=True).from_file(localFile)
        try:
            storage_client.upload_file(localFile, storageFile, fType)
        except:
            print('Error copying %s to storage' % fName)
            cleanDir = False
        else: 
            os.remove(localFile)
            
    print('All files in the working directory have been moved to the storage area directory:')
    print(storageDir)
    os.chdir(homeDir)
    if cleanDir:
        os.rmdir(useCaseDir)